In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import requests
import json

import time

## Support Functions

In [3]:
def get_keys(path):
    """
    Pulls necessary api keys from designated path
    """
    with open(path) as f:
        return json.load(f)

## ETL

In [11]:
current_congress = 116

### Voteview

In [17]:
def get_voteview_csv_content(content_type, current_congress_num):
    '''
    Retrieves congress data by content type (passed in) for the 104th Congress to the current Congress today.
    
    '''
    if content_type not in ['votes', 'rollcalls', 'members']:
        print('Not a valid content type')
        return
    
    for congress_num in range(104, current_congress_num+1):
        #download the vote each member made for each roll call
        url = 'https://voteview.com/static/data/out/{}/S{}_{}.csv'.format(content_type, congress_num, content_type)

        r = requests.get(url)
        content = r.content
        file = open('{}/S{}_{}.csv'.format(content_type, congress_num, content_type), 'wb')

        file.write(content)
        file.close()
        
        #put in a sleep timer to not overload the voteview servers
        time.sleep(1)
        
    return

In [18]:
get_voteview_csv_content('votes', current_congress)

In [19]:
get_voteview_csv_content('rollcalls', current_congress)

In [20]:
get_voteview_csv_content('members', current_congress)

### Open Secrets

### MIT Election Lab

### ProPublica

In [ ]:
key_path = "/Users/flatironschool/.secret/pro_publica_api.json"
keys = get_keys(key_path)

api_key = keys['api_key']

In [ ]:
version = 'v1'

url = 'https://api.propublica.org/congress/{}/'.format(version)

"https://api.propublica.org/congress/v1/116/bills/hr502.json"

variables = ['DATE_CODE', 'DATE_DESC', 'POP', 'NAME']

params = {'get': ','.join(variables), 'for': granularity, 'key': api_key}

r = requests.get(url, params=params)
print(r.url)
print(r)
print(r.text[:1000])